In [34]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from acquire import acquire_exoplanet_data as aed
from wrangle import initial_clean
from wrangle import splitter
import explore

from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler
import scipy.stats as stats

In [2]:
df = aed()
df.head()

,hostname,pl_letter,sy_snum,sy_pnum,cb_flag,discoverymethod,disc_year,glat,glon,pl_orbper,...,st_teff,st_met,st_lum,st_logg,st_age,st_mass,st_dens,st_rad,sy_dist,rowupdate
0,Kepler-11,c,1,6,0,Transit,2010,8.13328,76.16130,13.025020,...,5680.0,0.00,NaN,4.30,8.0,0.95,NaN,1.10,646.3460,2014-05-16
1,Kepler-11,f,1,6,0,Transit,2010,8.13328,76.16130,46.688760,...,5680.0,0.00,NaN,4.30,8.0,0.95,NaN,1.10,646.3460,2014-05-16
2,OGLE-TR-10,b,1,1,0,Transit,2004,-1.57666,359.85149,3.101278,...,5950.0,NaN,0.188,NaN,NaN,1.14,NaN,1.17,1344.9700,2014-05-14
3,HD 210702,b,1,1,0,Radial Velocity,2007,-32.02111,76.29345,354.290000,...,NaN,NaN,NaN,NaN,NaN,1.85,NaN,NaN,54.1963,2018-12-03
4,BD-08 2823,b,1,2,0,Radial Velocity,2009,34.75486,248.49513,5.600000,...,4746.0,-0.07,NaN,4.13,4.5,0.74,NaN,NaN,41.3342,2018-04-25


In [3]:
df = initial_clean(df)
df.head()

,num_planets_in_sys,orbital_period,y,star_temp,metallicity,luminosity,star_age,star_mass,star_density,star_radius,star_distance_from_earth,multistar,discovery_order,y_encoded
planet,,,,,,,,,,,,,,
14 Her-b,2,1766.601670,Not-Earthlike,5296.985000,0.415000,-0.1515,3.90,0.930000,1.2739,0.976667,17.9323,0,1,0
51 Peg-b,1,4.230530,Not-Earthlike,5763.920000,0.195333,0.1360,10.00,1.046667,0.8937,1.220000,15.4614,0,1,0
55 Cnc-b,5,14.632669,Not-Earthlike,5241.412857,0.329125,-0.1970,5.25,0.918000,1.4185,0.956667,12.5855,1,1,0
55 Cnc-d,5,5000.560000,Not-Earthlike,5248.315000,0.343286,-0.1970,5.50,0.925556,1.4185,0.956667,12.5855,1,3,0
BD+20 594-b,1,41.686548,Not-Earthlike,5745.000000,-0.120000,-0.0560,3.34,1.196667,1.7950,0.990000,179.4610,0,1,0


### Cleaned, now time to split

In [5]:
train, validate, test = splitter(df)
train.head(2)

Train = 1579 rows (56.0%) | Validate = 677 rows (24.0%) | Test = 564 rows (20.0%)
You did not stratify.  If looking to stratify, ensure to add argument: "target = variable to stratify on".


,num_planets_in_sys,orbital_period,y,star_temp,metallicity,luminosity,star_age,star_mass,star_density,star_radius,star_distance_from_earth,multistar,discovery_order,y_encoded
planet,,,,,,,,,,,,,,
Kepler-317-b,2,5.524177,Not-Earthlike,5445.60,0.2140,-0.20800,7.66,0.942222,1.868017,0.938182,940.584,0,1,0
Kepler-220-c,4,9.034198,Not-Earthlike,4589.74,-0.1665,-0.68725,7.16,0.702000,10.347129,0.699167,171.109,0,2,0


### Creating clusters of luminosity and star temp, which replicates the diagram for a standard main sequence stars.  Will give info on main sequence stars!

In [7]:
X_lt_train = train[['luminosity','star_temp']]
X_lt_validate = validate[['luminosity','star_temp']]
X_lt_test = test[['luminosity','star_temp']]

kmeans = KMeans(n_clusters=4)
kmeans.fit(X_lt_train)

train['lt_cluster'] = kmeans.predict(X_lt_train)
validate['lt_cluster'] = kmeans.predict(X_lt_validate)
test['lt_cluster'] = kmeans.predict(X_lt_test)

test.head(2)

/var/folders/nw/8yyyyp8n70vbplhdsylyqs8c0000gn/T/ipykernel_38489/1519378505.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['lt_cluster'] = kmeans.predict(X_lt_test)


,num_planets_in_sys,orbital_period,y,star_temp,metallicity,luminosity,star_age,star_mass,star_density,star_radius,star_distance_from_earth,multistar,discovery_order,y_encoded,lt_cluster
planet,,,,,,,,,,,,,,,
Kepler-284-c,2,37.514155,Not-Earthlike,5468.636364,-0.060,-0.341,2.485,0.914444,1.895800,0.816364,1031.950,0,2,0,2
Kepler-732-b,2,9.467820,Not-Earthlike,3622.900000,0.118,-1.375,9.085,0.492222,5.754429,0.483000,174.826,0,1,0,0


## *This is where a quick Q&A needs to happen around star distance from earth - is there a difference in mean between those discovered and those not!*

Null Hypothesis: The mean distance to earthlike planets is the same or greater than the mean to any planet
Alternate Hypothesis: The mean distance to earthlike planets is less than the mean to any planet.

In [16]:
train.groupby('y').star_distance_from_earth.mean()

y
Earthlike        603.772665
Not-Earthlike    746.669534
Name: star_distance_from_earth, dtype: float64

These means are so different it seems obvious, but I wanted to try and really test this one hard, so rather than compare the earthlike to the mean of the non-earthlike, but to the mean of all exoplanets.

In [17]:
earthlike_dist_mean = train.groupby('y').star_distance_from_earth.mean()[0]
earthlike_dist_mean

603.7726652631579

In [18]:
dist_mean = train.star_distance_from_earth.mean()
dist_mean

738.0721924635845

Varainces Check

In [23]:
a = train[train.y == 'Earthlike']
b = train[train.y == 'Not-Earthlike']
print(f'Variances = {a.star_distance_from_earth.var()} and {b.star_distance_from_earth.var()}\n-----\n')

Variances = 113523.76042414342 and 225687.51941902816
-----



I guess that is big, so I will use a Unequal Variance attribute.
I will use a one-sample, one-tail t-test (signifigantly smaller)

In [27]:
t, p = stats.ttest_1samp(a.star_distance_from_earth, b.star_distance_from_earth.mean())

print(t, p/2)

-4.13372071494515 3.8752573292419e-05


Reject the null.  Closer is better.  Good to know for future analysis.

### Drop unecessary columns (either captured in clustering or other features)
1. Clustering captures luminosity, and temp, 
2. Density captures mass and radius (which happens to be related to temperature and luminosity anyway)
3. Star distance from earth no longer needed - above analysis noted

In [29]:
train = train.drop(columns = ['luminosity','star_temp','star_mass','star_radius','star_distance_from_earth'])
validate = validate.drop(columns = ['luminosity','star_temp','star_mass','star_radius','star_distance_from_earth'])
test = test.drop(columns = ['luminosity','star_temp','star_mass','star_radius','star_distance_from_earth'])

Scale or not scale numericals?

In [33]:
train.describe().T

,count,mean,std,min,25%,50%,75%,max
num_planets_in_sys,1579.0,1.816973,1.154180,1.000000,1.000000,1.000000,2.000000,8.000000
orbital_period,1579.0,50.209052,311.143829,0.355005,4.434598,10.087456,24.149268,8241.000000
metallicity,1579.0,-0.030742,0.183431,-1.080875,-0.136500,-0.029625,0.080000,0.522000
star_age,1579.0,4.541407,2.926154,0.017000,2.385000,3.833333,6.353333,14.900000
star_density,1579.0,2.707469,4.396446,0.060043,0.777588,1.637229,2.834056,58.267417
multistar,1579.0,0.058265,0.234318,0.000000,0.000000,0.000000,0.000000,1.000000
discovery_order,1579.0,1.377454,0.742959,1.000000,1.000000,1.000000,2.000000,6.000000
y_encoded,1579.0,0.060165,0.237867,0.000000,0.000000,0.000000,0.000000,1.000000
lt_cluster,1579.0,1.782141,0.899604,0.000000,1.000000,2.000000,2.000000,3.000000


We'll scale orbital_period and star_density, everything else looks ok.

In [38]:
train[['orbital_period','star_density']]

,orbital_period,star_density
planet,,
Kepler-317-b,5.524177,1.868017
Kepler-220-c,9.034198,10.347129
Kepler-119-c,4.125106,1.373920
Kepler-271-c,7.410887,2.187771
Kepler-125-c,5.774478,6.170586
...,...,...
Kepler-1119-b,8.326484,0.363486
Kepler-626-b,14.485858,3.385600
Kepler-1149-b,3.730905,0.807483


In [42]:
scale_cols = ['orbital_period','star_density']

scaler = MinMaxScaler()
scaler.fit(train[scale_cols])

train_scaled_features = pd.DataFrame(scaler.transform(train[scale_cols]),columns=['scaled_orb_period','scaled_star_density']).set_index([train.index.values])
train_scaled_features

,scaled_orb_period,scaled_star_density
Kepler-317-b,0.000627,0.031061
Kepler-220-c,0.001053,0.176732
Kepler-119-c,0.000458,0.022572
Kepler-271-c,0.000856,0.036554
Kepler-125-c,0.000658,0.104979
...,...,...
Kepler-1119-b,0.000967,0.005213
Kepler-626-b,0.001715,0.057133
Kepler-1149-b,0.000410,0.012841
Kepler-1556-b,0.001028,0.100237


In [47]:
validate_scaled_features = pd.DataFrame(scaler.transform(validate[scale_cols]),columns=['scaled_orb_period','scaled_star_density']).set_index([validate.index.values])
test_scaled_features = pd.DataFrame(scaler.transform(test[scale_cols]),columns=['scaled_orb_period','scaled_star_density']).set_index([test.index.values])

In [67]:
train_scaled = train.merge(train_scaled_features, left_index = True, right_index = True).drop(columns = scale_cols)
validate_scaled = validate.merge(validate_scaled_features, left_index = True, right_index = True).drop(columns = scale_cols)
test_scaled = test.merge(test_scaled_features, left_index = True, right_index = True).drop(columns = scale_cols)

In [74]:
X_train = train_scaled.drop(columns=['y','y_encoded'])
y_train = train_scaled.y

X_validate = validate_scaled.drop(columns=['y','y_encoded'])
y_validate = validate_scaled.y

X_test = test_scaled.drop(columns=['y','y_encoded'])
y_test = test_scaled.y

In [75]:
X_train.sample(5)

,num_planets_in_sys,metallicity,star_age,multistar,discovery_order,lt_cluster,scaled_orb_period,scaled_star_density
Kepler-1387-b,1,0.028286,9.185,0,1,3,0.000234,0.060048
HAT-P-22-b,1,0.255714,11.550,1,1,3,0.000347,0.015642
Kepler-1057-b,1,-0.078000,1.745,0,1,2,0.001667,0.030798
Kepler-192-c,3,0.127625,3.630,0,2,2,0.002532,0.012463
HAT-P-66-b,1,0.035000,4.660,0,1,1,0.000318,0.003590


### Now we have our modeling sets!

First thing to do: set baseline

In [76]:
# Calculate baseline accuracy pecentage
baseline = y_train.mode()
matches_baseline_prediction = (y_train == baseline.values[0])
baseline_accuracy = matches_baseline_prediction.mean()
print(f"Baseline accuracy: {round(baseline_accuracy, 2)}")

Baseline accuracy: 0.94


#### This is pretty high, and its because by far most planets are NOT earthlike!  Because of this, accuracy is probably not as important as Sensitivity (Recall) as False Negatives are better than False Positives, because we don't want to waste respources going after true negatives.

In [88]:
from matrix_result import matrix_result_2d
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression

## Decision Tree - k from 2 to 10

In [84]:
# Valditing decision tree model with selected hyperparameters on out-of-sample data
for k in range (2,11):
    print(f'k={k}')
    tree = DecisionTreeClassifier(max_depth=k, random_state=123)
    tree = tree.fit(X_train, y_train)
    y_tree_pred = tree.predict(X_validate)
    result = pd.DataFrame(confusion_matrix(y_validate, y_tree_pred))
    results = matrix_result_2d(result)
    results['model'] = ('Decision Tree, Depth =',k)
    matrix_result_2d(result, output='print')
    print('\n-----\n')
    

k=2
True Positive = 0
False Positive = 0
True Negative = 636
False Negative = 41
Precision = nan
Recall = 0.00
Accuracy = 0.94
F1 Score = nan
Support, Positive = 41
Support, Negative = 636

-----

k=3
True Positive = 0
False Positive = 0
True Negative = 636
False Negative = 41
Precision = nan
Recall = 0.00
Accuracy = 0.94
F1 Score = nan
Support, Positive = 41
Support, Negative = 636

-----

k=4
True Positive = 0
False Positive = 0
True Negative = 636
False Negative = 41
Precision = nan
Recall = 0.00
Accuracy = 0.94
F1 Score = nan
Support, Positive = 41
Support, Negative = 636

-----

k=5
True Positive = 0
False Positive = 1
True Negative = 635
False Negative = 41
Precision = 0.00
Recall = 0.00
Accuracy = 0.94
F1 Score = nan
Support, Positive = 41
Support, Negative = 636

-----

k=6
True Positive = 0
False Positive = 3
True Negative = 633
False Negative = 41
Precision = 0.00
Recall = 0.00
Accuracy = 0.94
F1 Score = nan
Support, Positive = 41
Support, Negative = 636

-----

k=7
True Posi

/Users/ryanmiller-alt/codeup-data-science/exoplanet-analysis/matrix_result.py:39: RuntimeWarning: invalid value encountered in long_scalars
  precision = true_positive / (true_positive + false_positive)
/Users/ryanmiller-alt/codeup-data-science/exoplanet-analysis/matrix_result.py:39: RuntimeWarning: invalid value encountered in long_scalars
  precision = true_positive / (true_positive + false_positive)
/Users/ryanmiller-alt/codeup-data-science/exoplanet-analysis/matrix_result.py:39: RuntimeWarning: invalid value encountered in long_scalars
  precision = true_positive / (true_positive + false_positive)
/Users/ryanmiller-alt/codeup-data-science/exoplanet-analysis/matrix_result.py:42: RuntimeWarning: invalid value encountered in double_scalars
  f1_score = (2*precision*recall)/(precision + recall)
/Users/ryanmiller-alt/codeup-data-science/exoplanet-analysis/matrix_result.py:42: RuntimeWarning: invalid value encountered in double_scalars
  f1_score = (2*precision*recall)/(precision + recal

k = 8 is probably the best, although not really usable.

## Random Forest, Depth 2-7, Leaves 3-8

In [91]:
for d in range (1,3):
    for l in range (3,9):
        print(f'd = {d} l = {l}')
        forest = RandomForestClassifier(max_depth=d, min_samples_leaf = l,random_state=123)
        forest = forest.fit(X_train, y_train)
        y_forest_pred = forest.predict(X_validate)
        result = pd.DataFrame(confusion_matrix(y_validate, y_forest_pred))
        results = matrix_result_2d(result)
        results['model'] = ('Random Forest, Depth =',d,'Leaves =',l)
        matrix_result_2d(result, output='print')
        print('\n-----\n')

d = 1 l = 3
True Positive = 0
False Positive = 0
True Negative = 636
False Negative = 41
Precision = nan
Recall = 0.00
Accuracy = 0.94
F1 Score = nan
Support, Positive = 41
Support, Negative = 636

-----

d = 1 l = 4
True Positive = 0
False Positive = 0
True Negative = 636
False Negative = 41
Precision = nan
Recall = 0.00
Accuracy = 0.94
F1 Score = nan
Support, Positive = 41
Support, Negative = 636

-----

d = 1 l = 5


/Users/ryanmiller-alt/codeup-data-science/exoplanet-analysis/matrix_result.py:39: RuntimeWarning: invalid value encountered in long_scalars
  precision = true_positive / (true_positive + false_positive)
/Users/ryanmiller-alt/codeup-data-science/exoplanet-analysis/matrix_result.py:39: RuntimeWarning: invalid value encountered in long_scalars
  precision = true_positive / (true_positive + false_positive)
/Users/ryanmiller-alt/codeup-data-science/exoplanet-analysis/matrix_result.py:39: RuntimeWarning: invalid value encountered in long_scalars
  precision = true_positive / (true_positive + false_positive)


True Positive = 0
False Positive = 0
True Negative = 636
False Negative = 41
Precision = nan
Recall = 0.00
Accuracy = 0.94
F1 Score = nan
Support, Positive = 41
Support, Negative = 636

-----

d = 1 l = 6
True Positive = 0
False Positive = 0
True Negative = 636
False Negative = 41
Precision = nan
Recall = 0.00
Accuracy = 0.94
F1 Score = nan
Support, Positive = 41
Support, Negative = 636

-----

d = 1 l = 7
True Positive = 0
False Positive = 0
True Negative = 636
False Negative = 41
Precision = nan
Recall = 0.00
Accuracy = 0.94
F1 Score = nan
Support, Positive = 41
Support, Negative = 636

-----

d = 1 l = 8


/Users/ryanmiller-alt/codeup-data-science/exoplanet-analysis/matrix_result.py:39: RuntimeWarning: invalid value encountered in long_scalars
  precision = true_positive / (true_positive + false_positive)
/Users/ryanmiller-alt/codeup-data-science/exoplanet-analysis/matrix_result.py:39: RuntimeWarning: invalid value encountered in long_scalars
  precision = true_positive / (true_positive + false_positive)
/Users/ryanmiller-alt/codeup-data-science/exoplanet-analysis/matrix_result.py:39: RuntimeWarning: invalid value encountered in long_scalars
  precision = true_positive / (true_positive + false_positive)


True Positive = 0
False Positive = 0
True Negative = 636
False Negative = 41
Precision = nan
Recall = 0.00
Accuracy = 0.94
F1 Score = nan
Support, Positive = 41
Support, Negative = 636

-----

d = 2 l = 3
True Positive = 0
False Positive = 0
True Negative = 636
False Negative = 41
Precision = nan
Recall = 0.00
Accuracy = 0.94
F1 Score = nan
Support, Positive = 41
Support, Negative = 636

-----

d = 2 l = 4
True Positive = 0
False Positive = 0
True Negative = 636
False Negative = 41
Precision = nan
Recall = 0.00
Accuracy = 0.94
F1 Score = nan
Support, Positive = 41
Support, Negative = 636

-----

d = 2 l = 5


/Users/ryanmiller-alt/codeup-data-science/exoplanet-analysis/matrix_result.py:39: RuntimeWarning: invalid value encountered in long_scalars
  precision = true_positive / (true_positive + false_positive)
/Users/ryanmiller-alt/codeup-data-science/exoplanet-analysis/matrix_result.py:39: RuntimeWarning: invalid value encountered in long_scalars
  precision = true_positive / (true_positive + false_positive)
/Users/ryanmiller-alt/codeup-data-science/exoplanet-analysis/matrix_result.py:39: RuntimeWarning: invalid value encountered in long_scalars
  precision = true_positive / (true_positive + false_positive)


True Positive = 0
False Positive = 0
True Negative = 636
False Negative = 41
Precision = nan
Recall = 0.00
Accuracy = 0.94
F1 Score = nan
Support, Positive = 41
Support, Negative = 636

-----

d = 2 l = 6
True Positive = 0
False Positive = 0
True Negative = 636
False Negative = 41
Precision = nan
Recall = 0.00
Accuracy = 0.94
F1 Score = nan
Support, Positive = 41
Support, Negative = 636

-----

d = 2 l = 7
True Positive = 0
False Positive = 0
True Negative = 636
False Negative = 41
Precision = nan
Recall = 0.00
Accuracy = 0.94
F1 Score = nan
Support, Positive = 41
Support, Negative = 636

-----

d = 2 l = 8
True Positive = 0
False Positive = 0
True Negative = 636
False Negative = 41
Precision = nan
Recall = 0.00
Accuracy = 0.94
F1 Score = nan
Support, Positive = 41
Support, Negative = 636

-----



/Users/ryanmiller-alt/codeup-data-science/exoplanet-analysis/matrix_result.py:39: RuntimeWarning: invalid value encountered in long_scalars
  precision = true_positive / (true_positive + false_positive)
/Users/ryanmiller-alt/codeup-data-science/exoplanet-analysis/matrix_result.py:39: RuntimeWarning: invalid value encountered in long_scalars
  precision = true_positive / (true_positive + false_positive)
/Users/ryanmiller-alt/codeup-data-science/exoplanet-analysis/matrix_result.py:39: RuntimeWarning: invalid value encountered in long_scalars
  precision = true_positive / (true_positive + false_positive)


None of these work.

## K-Nearest neighbors

In [92]:
for n in range (5,25):
    print(n)
    print('\n')
    knn = KNeighborsClassifier(n_neighbors = n)
    knn = knn.fit(X_train, y_train)
    y_knn_pred = knn.predict(X_validate)
    result = pd.DataFrame(confusion_matrix(y_validate, y_knn_pred))
    results = matrix_result_2d(result)
    results['model'] = ('KNN, Neighbors =',n)
    matrix_result_2d(result, output='print')
    print('\n-----\n')

5


True Positive = 0
False Positive = 2
True Negative = 634
False Negative = 41
Precision = 0.00
Recall = 0.00
Accuracy = 0.94
F1 Score = nan
Support, Positive = 41
Support, Negative = 636

-----

6


True Positive = 0
False Positive = 4
True Negative = 632
False Negative = 41
Precision = 0.00
Recall = 0.00
Accuracy = 0.93
F1 Score = nan
Support, Positive = 41
Support, Negative = 636

-----

7


True Positive = 0
False Positive = 0
True Negative = 636
False Negative = 41
Precision = nan
Recall = 0.00
Accuracy = 0.94
F1 Score = nan
Support, Positive = 41
Support, Negative = 636

-----

8


True Positive = 0
False Positive = 0
True Negative = 636
False Negative = 41
Precision = nan
Recall = 0.00
Accuracy = 0.94
F1 Score = nan
Support, Positive = 41
Support, Negative = 636

-----

9


True Positive = 0
False Positive = 0
True Negative = 636
False Negative = 41
Precision = nan
Recall = 0.00
Accuracy = 0.94
F1 Score = nan
Support, Positive = 41
Support, Negative = 636

-----

10


True Pos

/Users/ryanmiller-alt/codeup-data-science/exoplanet-analysis/matrix_result.py:42: RuntimeWarning: invalid value encountered in double_scalars
  f1_score = (2*precision*recall)/(precision + recall)
/Users/ryanmiller-alt/codeup-data-science/exoplanet-analysis/matrix_result.py:42: RuntimeWarning: invalid value encountered in double_scalars
  f1_score = (2*precision*recall)/(precision + recall)
/Users/ryanmiller-alt/codeup-data-science/exoplanet-analysis/matrix_result.py:39: RuntimeWarning: invalid value encountered in long_scalars
  precision = true_positive / (true_positive + false_positive)
/Users/ryanmiller-alt/codeup-data-science/exoplanet-analysis/matrix_result.py:39: RuntimeWarning: invalid value encountered in long_scalars
  precision = true_positive / (true_positive + false_positive)
/Users/ryanmiller-alt/codeup-data-science/exoplanet-analysis/matrix_result.py:39: RuntimeWarning: invalid value encountered in long_scalars
  precision = true_positive / (true_positive + false_positiv

True Positive = 0
False Positive = 0
True Negative = 636
False Negative = 41
Precision = nan
Recall = 0.00
Accuracy = 0.94
F1 Score = nan
Support, Positive = 41
Support, Negative = 636

-----

14


True Positive = 0
False Positive = 0
True Negative = 636
False Negative = 41
Precision = nan
Recall = 0.00
Accuracy = 0.94
F1 Score = nan
Support, Positive = 41
Support, Negative = 636

-----

15


True Positive = 0
False Positive = 0
True Negative = 636
False Negative = 41
Precision = nan
Recall = 0.00
Accuracy = 0.94
F1 Score = nan
Support, Positive = 41
Support, Negative = 636

-----

16


True Positive = 0
False Positive = 0
True Negative = 636
False Negative = 41
Precision = nan
Recall = 0.00
Accuracy = 0.94
F1 Score = nan
Support, Positive = 41
Support, Negative = 636

-----

17


True Positive = 0
False Positive = 0
True Negative = 636
False Negative = 41
Precision = nan
Recall = 0.00
Accuracy = 0.94
F1 Score = nan
Support, Positive = 41
Support, Negative = 636

-----

18


True Posit

/Users/ryanmiller-alt/codeup-data-science/exoplanet-analysis/matrix_result.py:39: RuntimeWarning: invalid value encountered in long_scalars
  precision = true_positive / (true_positive + false_positive)
/Users/ryanmiller-alt/codeup-data-science/exoplanet-analysis/matrix_result.py:39: RuntimeWarning: invalid value encountered in long_scalars
  precision = true_positive / (true_positive + false_positive)
/Users/ryanmiller-alt/codeup-data-science/exoplanet-analysis/matrix_result.py:39: RuntimeWarning: invalid value encountered in long_scalars
  precision = true_positive / (true_positive + false_positive)
/Users/ryanmiller-alt/codeup-data-science/exoplanet-analysis/matrix_result.py:39: RuntimeWarning: invalid value encountered in long_scalars
  precision = true_positive / (true_positive + false_positive)
/Users/ryanmiller-alt/codeup-data-science/exoplanet-analysis/matrix_result.py:39: RuntimeWarning: invalid value encountered in long_scalars
  precision = true_positive / (true_positive + f

Ok, this one didn't seem to do much either.

## Logistical Regression

In [93]:
# Valditing loistical regression model on out-of-sample data
logit = LogisticRegression()
logit = logit.fit(X_train, y_train)
y_logit_pred = logit.predict(X_validate)
result = pd.DataFrame(confusion_matrix(y_validate, y_logit_pred))
results = matrix_result_2d(result)
results['model'] = 'Logistical Regression, Default'
matrix_result_2d(result, output='print')

True Positive = 0
False Positive = 0
True Negative = 636
False Negative = 41
Precision = nan
Recall = 0.00
Accuracy = 0.94
F1 Score = nan
Support, Positive = 41
Support, Negative = 636


/Users/ryanmiller-alt/codeup-data-science/exoplanet-analysis/matrix_result.py:39: RuntimeWarning: invalid value encountered in long_scalars
  precision = true_positive / (true_positive + false_positive)


## Well, I don't think with the data and/or features/features set as-is, we are doing a great job of classifying which stars have earthlike planets.  That's ok.  We've got James Webb coming online!